# RAGAS 테스트 03 - 멀티턴 대화 평가

이 노트북은 멀티턴 대화 기능의 성능을 RAGAS로 평가합니다.


In [ ]:
import requests
import json
import pandas as pd
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall
)
from datasets import Dataset

print("RAGAS 테스트 03 - 멀티턴 대화 평가 시작")


In [ ]:
# 멀티턴 대화 테스트
def test_multiturn_conversation(questions, session_id=None):
    """멀티턴 대화를 테스트하는 함수"""
    url = "http://localhost:8001/api/v1/langgraph/multiturn"
    results = []
    
    for i, question in enumerate(questions):
        print(f"턴 {i+1}/{len(questions)}: {question[:50]}...")
        
        payload = {
            "prompt": question,
            "session_id": session_id
        }
        
        try:
            response = requests.post(url, json=payload)
            if response.status_code == 200:
                data = response.json()
                result = {
                    "question": question,
                    "answer": data.get("response", ""),
                    "sources": data.get("sources", []),
                    "session_id": data.get("session_info", {}).get("session_id"),
                    "conversation_mode": data.get("conversation_mode", ""),
                    "current_topic": data.get("current_topic", "")
                }
                results.append(result)
                session_id = result["session_id"]  # 다음 턴에서 같은 세션 사용
            else:
                print(f"API 호출 실패: {response.status_code}")
        except Exception as e:
            print(f"API 호출 중 오류: {str(e)}")
    
    return results

# 멀티턴 대화 시나리오
conversation_scenarios = [
    {
        "name": "상생대출 상담",
        "questions": [
            "KB 동반성장협약 상생대출에 대해 알려주세요",
            "그 대출의 신청 조건은 무엇인가요?",
            "금리는 어떻게 되나요?",
            "신청 방법을 알려주세요"
        ]
    },
    {
        "name": "스마트론 상담",
        "questions": [
            "KB 스마트론이 무엇인가요?",
            "신청 자격은 어떻게 되나요?",
            "한도는 얼마까지 가능한가요?",
            "온라인 신청이 가능한가요?"
        ]
    }
]

print(f"멀티턴 대화 시나리오 {len(conversation_scenarios)}개 준비 완료")


In [ ]:
# 멀티턴 대화 테스트 실행
all_results = []

for scenario in conversation_scenarios:
    print(f"\\n=== {scenario['name']} 시나리오 테스트 ===")
    results = test_multiturn_conversation(scenario['questions'])
    all_results.extend(results)
    print(f"{scenario['name']} 완료: {len(results)}개 턴")

print(f"\\n전체 멀티턴 대화 테스트 완료: {len(all_results)}개 턴")


In [ ]:
# RAGAS 평가를 위한 데이터 준비
ragas_data = {
    "question": [r["question"] for r in all_results],
    "answer": [r["answer"] for r in all_results],
    "contexts": [[source.get("text", "") for source in r["sources"]] for r in all_results],
    "ground_truths": [
        # 상생대출 관련
        ["KB 동반성장협약 상생대출은 중소기업 지원 대출"],
        ["상생대출 신청 조건은 중소기업 자격 요건"],
        ["상생대출 금리는 우대 금리 적용"],
        ["상생대출 신청은 온라인 또는 지점 방문"],
        # 스마트론 관련
        ["KB 스마트론은 온라인 간편 대출"],
        ["스마트론 신청 자격은 개인 신용 기준"],
        ["스마트론 한도는 개인별 차등"],
        ["스마트론은 온라인 신청 가능"]
    ]
}

dataset = Dataset.from_dict(ragas_data)
print(f"멀티턴 대화 RAGAS 평가 데이터셋 생성: {len(dataset)}개 샘플")


In [ ]:
# RAGAS 평가 실행
print("멀티턴 대화 RAGAS 평가 시작...")
result = evaluate(
    dataset,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall
    ]
)

print("\\n=== 멀티턴 대화 RAGAS 평가 결과 ===")
print(result)
